In [0]:

# MAGIC %md
# MAGIC # 03 - Model Evaluation & Analysis
# MAGIC 
# MAGIC **Comprehensive evaluation of the best model from MLflow**
# MAGIC 
# MAGIC ## Objectives:
# MAGIC - Load best model from MLflow Model Registry
# MAGIC - Detailed performance analysis
# MAGIC - Advanced visualizations
# MAGIC - Feature importance analysis
# MAGIC - Residual analysis
# MAGIC - Generate evaluation report


In [0]:
# MAGIC %md
# MAGIC ## 1. Setup & Imports


In [0]:
%restart_python

In [0]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# MLflow
import mlflow
import mlflow.sklearn

# Sklearn
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score,
    mean_absolute_percentage_error
)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports complete")


In [0]:

# MAGIC %md
# MAGIC ## 2. Project Setup


In [0]:

print("="*60)
print("PROJECT SETUP")
print("="*60)

# Define project root
project_root = "/Workspace/COMM - Commercial Analytics (CMAN)/MMM Quattro 2025/Satish/MLFLOW_sample"

# Add to path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"\n📂 Project root: {project_root}")
print(f"✅ Added to sys.path")

# Import custom modules
from src.utils import ConfigLoader, DataLoader, MLflowLogger, safe_display
from src.data_processing import DataProcessor
from src.model import ModelEvaluator

print(f"✅ Custom modules imported")
print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 3. Load Configuration


In [0]:

print("="*60)
print("LOADING CONFIGURATION")
print("="*60)

config_path = f'{project_root}/config/config.yaml'
config = ConfigLoader.load_config(config_path)

print(f"\n✅ Configuration loaded")
print(f"  • Project: {config['project']['name']}")
print(f"  • Model Registry: {config['mlflow']['model_registry_name']}")
print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 4. Setup MLflow


In [0]:

print("="*60)
print("SETTING UP MLFLOW")
print("="*60)

experiment_name = config['mlflow']['experiment_name']
mlflow.set_experiment(experiment_name)

print(f"\n✅ MLflow experiment set: {experiment_name}")

experiment = mlflow.get_experiment_by_name(experiment_name)
print(f"  • Experiment ID: {experiment.experiment_id}")
print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 5. Load Test Data


In [0]:

print("="*60)
print("LOADING TEST DATA")
print("="*60)

processed_path = config['data']['processed_path']

# Load test data
X_test = pd.read_csv(f"{processed_path}X_test.csv")
y_test = pd.read_csv(f"{processed_path}y_test.csv").squeeze()

print(f"\n✅ Test data loaded")
print(f"  • X_test shape: {X_test.shape}")
print(f"  • y_test shape: {y_test.shape}")

print(f"\n📊 Test Data Preview:")
safe_display(X_test)

print("="*60)

In [0]:

# MAGIC %md
# MAGIC ## 6. Load Best Model from MLflow


In [0]:

print("="*60)
print("LOADING BEST MODEL FROM MLFLOW")
print("="*60)

model_name = config['mlflow']['model_registry_name']

try:
    # Load latest version from registry
    model_uri = f"models:/{model_name}/latest"
    
    print(f"\n📦 Loading model: {model_uri}")
    
    best_model = mlflow.sklearn.load_model(model_uri)
    
    print(f"\n✅ Model loaded successfully!")
    print(f"  • Model Name: {model_name}")
    print(f"  • Model Type: {type(best_model).__name__}")
    
    # Get model version info
    from mlflow.tracking import MlflowClient
    client = MlflowClient()
    
    model_versions = client.search_model_versions(f"name='{model_name}'")
    latest_version = model_versions[0]
    
    print(f"\n📋 Model Version Info:")
    print(f"  • Version: {latest_version.version}")
    print(f"  • Stage: {latest_version.current_stage}")
    print(f"  • Run ID: {latest_version.run_id}")
    
except Exception as e:
    print(f"\n❌ Error loading model: {e}")
    print(f"\n💡 Tip: Make sure you ran the model training notebook first!")
    raise

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 7. Make Predictions


In [0]:

print("="*60)
print("MAKING PREDICTIONS")
print("="*60)

# Make predictions
y_pred = best_model.predict(X_test)

print(f"\n✅ Predictions complete")
print(f"  • Number of predictions: {len(y_pred)}")

# Create predictions DataFrame
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Residual': y_test - y_pred,
    'Abs_Error': np.abs(y_test - y_pred),
    'Pct_Error': np.abs((y_test - y_pred) / y_test) * 100
})

print(f"\n📊 Predictions Sample:")
safe_display(predictions_df.head(10))

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 8. Calculate Metrics


In [0]:

print("="*60)
print("CALCULATING METRICS")
print("="*60)

# Calculate comprehensive metrics
metrics = {
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
    'MAE': mean_absolute_error(y_test, y_pred),
    'R2': r2_score(y_test, y_pred),
    'MAPE': mean_absolute_percentage_error(y_test, y_pred) * 100,
    'Max_Error': predictions_df['Abs_Error'].max(),
    'Mean_Residual': predictions_df['Residual'].mean(),
    'Std_Residual': predictions_df['Residual'].std()
}

print(f"\n📊 Model Performance Metrics:")
print(f"  • RMSE:           ${metrics['RMSE']:,.2f}")
print(f"  • MAE:            ${metrics['MAE']:,.2f}")
print(f"  • R² Score:       {metrics['R2']:.4f}")
print(f"  • MAPE:           {metrics['MAPE']:.2f}%")
print(f"  • Max Error:      ${metrics['Max_Error']:,.2f}")
print(f"  • Mean Residual:  ${metrics['Mean_Residual']:,.2f}")
print(f"  • Std Residual:   ${metrics['Std_Residual']:,.2f}")

# Log metrics to MLflow
with mlflow.start_run(run_name="model_evaluation"):
    MLflowLogger.log_metrics_from_dict({
        f"eval_{k.lower()}": v for k, v in metrics.items()
    })
    print(f"\n✅ Metrics logged to MLflow")

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 9. Visualizations


In [0]:

# MAGIC %md
# MAGIC ### 9.1 Actual vs Predicted


In [0]:

print("="*60)
print("VISUALIZATION: ACTUAL VS PREDICTED")
print("="*60)

fig, ax = plt.subplots(figsize=(10, 6))

# Scatter plot
ax.scatter(y_test, y_pred, alpha=0.6, edgecolors='k', linewidth=0.5, s=50)

# Perfect prediction line
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

# Add R² to plot
ax.text(0.05, 0.95, f'R² = {metrics["R2"]:.4f}', 
        transform=ax.transAxes, fontsize=12, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

ax.set_xlabel('Actual Price ($)', fontsize=12, fontweight='bold')
ax.set_ylabel('Predicted Price ($)', fontsize=12, fontweight='bold')
ax.set_title('Actual vs Predicted House Prices', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualization complete")
print("="*60)


In [0]:

# MAGIC %md
# MAGIC ### 9.2 Residual Analysis


In [0]:

print("="*60)
print("VISUALIZATION: RESIDUAL ANALYSIS")
print("="*60)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Residuals vs Predicted
axes[0, 0].scatter(y_pred, predictions_df['Residual'], alpha=0.6, edgecolors='k', linewidth=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--', lw=2)
axes[0, 0].set_xlabel('Predicted Price ($)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Residuals ($)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Residual Plot', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Residuals Distribution
axes[0, 1].hist(predictions_df['Residual'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(x=0, color='r', linestyle='--', lw=2)
axes[0, 1].set_xlabel('Residuals ($)', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Residuals Distribution', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Absolute Error Distribution
axes[1, 0].hist(predictions_df['Abs_Error'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[1, 0].set_xlabel('Absolute Error ($)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Absolute Error Distribution', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Percentage Error Distribution
axes[1, 1].hist(predictions_df['Pct_Error'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1, 1].set_xlabel('Percentage Error (%)', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Percentage Error Distribution', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("✅ Residual analysis complete")
print("="*60)


In [0]:

# MAGIC %md
# MAGIC ### 9.3 Error Analysis


In [0]:

print("="*60)
print("ERROR ANALYSIS")
print("="*60)

# Identify best and worst predictions
best_predictions = predictions_df.nsmallest(5, 'Abs_Error')
worst_predictions = predictions_df.nlargest(5, 'Abs_Error')

print(f"\n🎯 Best Predictions (Lowest Error):")
safe_display(best_predictions)

print(f"\n⚠️ Worst Predictions (Highest Error):")
safe_display(worst_predictions)

# Error statistics by price range
predictions_df['Price_Range'] = pd.cut(predictions_df['Actual'], bins=5)
error_by_range = predictions_df.groupby('Price_Range').agg({
    'Abs_Error': ['mean', 'median', 'max'],
    'Pct_Error': ['mean', 'median']
}).round(2)

print(f"\n📊 Error Analysis by Price Range:")
safe_display(error_by_range)

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 10. Feature Importance


In [0]:

print("="*60)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Check if model has feature importance
if hasattr(best_model, 'feature_importances_'):
    # Tree-based models
    feature_importance = pd.DataFrame({
        'Feature': X_test.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print(f"\n📊 Feature Importance:")
    safe_display(feature_importance)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'])
    plt.xlabel('Importance', fontsize=12, fontweight='bold')
    plt.ylabel('Feature', fontsize=12, fontweight='bold')
    plt.title('Feature Importance', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
elif hasattr(best_model, 'coef_'):
    # Linear models
    coefficients = pd.DataFrame({
        'Feature': X_test.columns,
        'Coefficient': best_model.coef_
    }).sort_values('Coefficient', key=abs, ascending=False)
    
    print(f"\n📊 Model Coefficients:")
    safe_display(coefficients)
    
    # Plot coefficients
    plt.figure(figsize=(10, 6))
    colors = ['green' if c > 0 else 'red' for c in coefficients['Coefficient']]
    plt.barh(coefficients['Feature'], coefficients['Coefficient'], color=colors, alpha=0.7)
    plt.xlabel('Coefficient', fontsize=12, fontweight='bold')
    plt.ylabel('Feature', fontsize=12, fontweight='bold')
    plt.title('Model Coefficients', fontsize=14, fontweight='bold')
    plt.axvline(x=0, color='black', linestyle='--', lw=1)
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
else:
    print("\n⚠️ Model does not have feature importance or coefficients")

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 11. Save Evaluation Results



In [0]:

print("="*60)
print("SAVING EVALUATION RESULTS")
print("="*60)

# Save predictions
predictions_path = f"{processed_path}model_predictions.csv"
predictions_df.to_csv(predictions_path, index=False)
print(f"\n✅ Predictions saved: {predictions_path}")

# Save metrics
metrics_df = pd.DataFrame([metrics])
metrics_path = f"{processed_path}evaluation_metrics.csv"
metrics_df.to_csv(metrics_path, index=False)
print(f"✅ Metrics saved: {metrics_path}")

# Save feature importance if available
if hasattr(best_model, 'feature_importances_'):
    importance_path = f"{processed_path}feature_importance.csv"
    feature_importance.to_csv(importance_path, index=False)
    print(f"✅ Feature importance saved: {importance_path}")
elif hasattr(best_model, 'coef_'):
    coef_path = f"{processed_path}model_coefficients.csv"
    coefficients.to_csv(coef_path, index=False)
    print(f"✅ Coefficients saved: {coef_path}")

print("="*60)

In [0]:

# MAGIC %md
# MAGIC ## 12. Evaluation Summary


In [0]:

print("="*60)
print("EVALUATION SUMMARY")
print("="*60)

print(f"\n🎯 Model: {model_name}")
print(f"📊 Model Type: {type(best_model).__name__}")

print(f"\n📈 Performance Metrics:")
print(f"  • RMSE:  ${metrics['RMSE']:,.2f}")
print(f"  • MAE:   ${metrics['MAE']:,.2f}")
print(f"  • R²:    {metrics['R2']:.4f}")
print(f"  • MAPE:  {metrics['MAPE']:.2f}%")

print(f"\n📁 Saved Files:")
print(f"  • Predictions: model_predictions.csv")
print(f"  • Metrics: evaluation_metrics.csv")
if hasattr(best_model, 'feature_importances_'):
    print(f"  • Feature Importance: feature_importance.csv")
elif hasattr(best_model, 'coef_'):
    print(f"  • Coefficients: model_coefficients.csv")

print(f"\n✅ Model evaluation complete!")
print("="*60)